In [24]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from tqdm import notebook
from pandarallel import pandarallel
tqdm.pandas(desc="progress")
pandarallel.initialize(progress_bar = True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [2]:
contract_main = pd.read_csv('hack_2023/contract_main_info.csv', sep=';')
contract_item = pd.read_csv('hack_2023/contract_item_info.csv', sep=';', on_bad_lines='skip')
contract_termination = pd.read_csv('hack_2023/contract_termination.csv', sep=';')
contract_improper_execution = pd.read_csv('hack_2023/contract_improper_execution.csv', sep=';')
complaint_info = pd.read_csv('hack_2023/complaint_info.csv', sep=';')
participation_statistic = pd.read_csv('hack_2023/participation_statistic.csv', sep=';')

C:\Users\Admin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
def to_str(row):
    try:
        return str(int(row))
    except:
        return row
def to_int(row):
    try:
        return int(row)
    except:
        return 0
contract_main['customer_inn'] = contract_main['customer_inn'].apply(to_str)
contract_main['supplier_inn'] = contract_main['supplier_inn'].apply(to_str)
contract_main['id_procedure'] = contract_main['id_procedure'].apply(to_int)
contract_main['execution_start_date'] =  pd.to_datetime(contract_main['execution_start_date'], format='%Y-%m-%d', errors='coerce')
contract_main['execution_end_date'] =  pd.to_datetime(contract_main['execution_end_date'], format='%Y-%m-%d', errors='coerce')
contract_main['execution_start_date'] = contract_main['execution_start_date'].apply(lambda x: x if not pd.isnull(x) else '')
contract_main['execution_end_date'] = contract_main['execution_end_date'].apply(lambda x: x if not pd.isnull(x) else '')
contract_main = contract_main.loc[~((contract_main['execution_end_date'] == '') 
                                    | (contract_main['execution_start_date'] == '') 
                                    | (contract_main['execution_start_date'].isna())
                                    | (contract_main['execution_end_date'].isna()))]
contract_main['time_for_work'] = ((contract_main['execution_end_date'] - 
                                   contract_main['execution_start_date']).dt.days.astype('int64'))
contract_main = contract_main.loc[contract_main['time_for_work'] < 401]

C:\Users\Admin\AppData\Local\Temp/ipykernel_8748/3156118301.py:16: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  contract_main['execution_start_date'] = contract_main['execution_start_date'].apply(lambda x: x if not pd.isnull(x) else '')
C:\Users\Admin\AppData\Local\Temp/ipykernel_8748/3156118301.py:17: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  contract_main['execution_end_date'] = contract_main['execution_end_date'].apply(lambda x: x if not pd.isnull(x) else '')


In [4]:
okpd = pd.read_excel('hack_2023/okpd.xlsx', sheet_name='Лист1')
okpd = okpd.drop(range(0, 7), axis=0)
okpd.columns = ['№', 'Код', 'Название']
okpd = okpd.drop(['№'], axis=1)

In [5]:
def get_data_info(data):
    display(data.sample(20))
    display(data.info())
    display(data.describe(include='all'))

In [6]:
code = list(okpd['Код'])
name = list(okpd['Название'])
dict_name_code = {}
for i in range(len(name)):
    dict_name_code[code[i]]=name[i]

In [7]:
def code_name(row):
    try:
        return dict_name_code[row]
    except:
        return 0
contract_item['tru_okpd2_name'] = contract_item['tru_okpd2_code'].apply(code_name)

In [8]:
contract_main = contract_main[['id_contract', 'id_procedure', 'customer_inn', 'sign_date', 'contract_price_rub', 'execution_start_date', 'execution_end_date', 'supplier_inn', 'time_for_work']]
contract_item = contract_item[['id_contract', 'tru_okpd2_name']]
contract_termination = contract_termination[['id_contract', 't_reason_name']]
contract_improper_execution = contract_improper_execution[['id_contract', 'e_doc_execution_name']]
complaint_info = complaint_info[['id_procedure', 'processing_result']]
contract_main.shape

(2526900, 9)

# Объединим информацию по контрактам

In [9]:
contract = contract_main.merge(contract_item, on='id_contract', how='outer')
contract = contract.merge(contract_termination, on='id_contract', how='outer')
contract = contract.merge(contract_improper_execution, on='id_contract', how='outer')
contract = contract.merge(complaint_info, on='id_procedure', how='outer')

contract_customer = contract
contract_customer = contract_customer.dropna(subset=['id_contract'])
contract_customer['id_contract'] = contract_customer['id_contract'].apply(to_int)
contract_supplier = contract
contract_supplier = contract_supplier.dropna(subset=['id_contract'])
contract_supplier['id_contract'] = contract_supplier['id_contract'].apply(to_int)
contract_customer.head()

C:\Users\Admin\AppData\Local\Temp/ipykernel_8748/1589979882.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contract_customer['id_contract'] = contract_customer['id_contract'].apply(to_int)
C:\Users\Admin\AppData\Local\Temp/ipykernel_8748/1589979882.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contract_supplier['id_contract'] = contract_supplier['id_contract'].apply(to_int)


,id_contract,id_procedure,customer_inn,sign_date,contract_price_rub,execution_start_date,execution_end_date,supplier_inn,time_for_work,tru_okpd2_name,t_reason_name,e_doc_execution_name,processing_result
0,11694128,0.0,7309900546,2019-09-23,120603.88,2019-09-23,2019-09-30,7327012462,7.0,"Электроэнергия, произведенная теплоэлектроцент...",NaN,NaN,NaN
1,6476238,0.0,2221037355,2019-03-25,66536.52,2019-01-01,2019-12-31,2225163706,364.0,Услуги по обслуживанию помещений комплексные,Соглашение сторон,NaN,NaN
2,6476238,0.0,2221037355,2019-03-25,66536.52,2019-01-01,2019-12-31,2225163706,364.0,Услуги по обслуживанию помещений комплексные,Соглашение сторон,NaN,NaN
3,8685287,0.0,3329010657,2019-03-26,330000.00,2019-03-26,2020-01-18,7704440018,298.0,Услуги по распределению электроэнергии,Соглашение сторон,NaN,NaN
4,8685287,0.0,3329010657,2019-03-26,330000.00,2019-03-26,2020-01-18,7704440018,298.0,Услуги по распределению электроэнергии,Соглашение сторон,NaN,NaN


In [10]:
contract_customer['processing_result'].unique()

array([nan, 'Признана необоснованной', 'Признана обоснованной',
       'Признана обоснованной (частично)'], dtype=object)

## Заказчики

Считаем количество жалоб

In [11]:
contract_customer = pd.get_dummies(contract_customer, columns=['processing_result'])
contract_customer.head()

,id_contract,id_procedure,customer_inn,sign_date,contract_price_rub,execution_start_date,execution_end_date,supplier_inn,time_for_work,tru_okpd2_name,t_reason_name,e_doc_execution_name,processing_result_Признана необоснованной,processing_result_Признана обоснованной,processing_result_Признана обоснованной (частично)
0,11694128,0.0,7309900546,2019-09-23,120603.88,2019-09-23,2019-09-30,7327012462,7.0,"Электроэнергия, произведенная теплоэлектроцент...",NaN,NaN,0,0,0
1,6476238,0.0,2221037355,2019-03-25,66536.52,2019-01-01,2019-12-31,2225163706,364.0,Услуги по обслуживанию помещений комплексные,Соглашение сторон,NaN,0,0,0
2,6476238,0.0,2221037355,2019-03-25,66536.52,2019-01-01,2019-12-31,2225163706,364.0,Услуги по обслуживанию помещений комплексные,Соглашение сторон,NaN,0,0,0
3,8685287,0.0,3329010657,2019-03-26,330000.00,2019-03-26,2020-01-18,7704440018,298.0,Услуги по распределению электроэнергии,Соглашение сторон,NaN,0,0,0
4,8685287,0.0,3329010657,2019-03-26,330000.00,2019-03-26,2020-01-18,7704440018,298.0,Услуги по распределению электроэнергии,Соглашение сторон,NaN,0,0,0


Обоснованные

In [12]:
reasonable_count = pd.pivot_table(contract_customer, values='processing_result_Признана обоснованной', index=['customer_inn'], aggfunc=np.sum)
contract_customer = contract_customer.merge(reasonable_count, on='customer_inn', how='outer')

Обоснованные (частично)

In [13]:
part_reasonable_count = pd.pivot_table(contract_customer, values='processing_result_Признана обоснованной (частично)', index=['customer_inn'], aggfunc=np.sum)
contract_customer = contract_customer.merge(part_reasonable_count, on='customer_inn', how='outer')

Необоснованные

In [14]:
unreasonable_count = pd.pivot_table(contract_customer, values='processing_result_Признана необоснованной', index=['customer_inn'], aggfunc=np.sum)
contract_customer = contract_customer.merge(unreasonable_count, on='customer_inn', how='outer')

Считаем количество проблемных расторженных контрактов для заказчика (судебный иск, односторонний отказ заказчика от исполнения)

In [15]:
contract_customer.loc[(contract_customer['t_reason_name'] == 'Соглашение сторон') | (contract_customer['t_reason_name'] == 'Односторонний отказ заказчика от исполнения контракта в соответствии с гражданским законодательством'), 't_reason_name'] = np.nan
table_t_reason_count = pd.pivot_table(contract_customer, values='t_reason_name', index=['customer_inn'], aggfunc='count')
contract_customer = contract_customer.drop('t_reason_name', axis=1)
contract_customer = contract_customer.merge(table_t_reason_count, on='customer_inn', how='outer')

Считаем стоимость всех контрактов для каждого инн

In [16]:
table_contract_price_rub = pd.pivot_table(contract_customer, values='contract_price_rub', index=['customer_inn'], aggfunc=np.sum)

Считаем кол-во контрактов для каждого инн

In [17]:
contract_count = pd.pivot_table(contract_customer, values='id_contract', index=['customer_inn'], aggfunc='count')
contract_customer = contract_customer.merge(contract_count, on='customer_inn', how='outer')

In [18]:
contract_customer = contract_customer.drop(['processing_result_Признана необоснованной_x', 'processing_result_Признана обоснованной_x', 'processing_result_Признана обоснованной (частично)_x'], axis=1)
contract_customer = contract_customer.rename(columns={'processing_result_Признана обоснованной_y':'reasonable_count',\
                                                     'processing_result_Признана обоснованной (частично)_y':'part_reasonable_count',\
                                                     'processing_result_Признана необоснованной_y':'unreasonable_count',\
                                                     'id_contract_y':'contract_count',\
                                                     'id_contract_x':'id_contract'})

contract_customer['sum_count_purposes'] = (contract_customer['reasonable_count'] +
                                          contract_customer['part_reasonable_count'] + 
                                          contract_customer['unreasonable_count'] +
                                          contract_customer['t_reason_name'])
contract_customer.head()

,id_contract,id_procedure,customer_inn,sign_date,contract_price_rub,execution_start_date,execution_end_date,supplier_inn,time_for_work,tru_okpd2_name,e_doc_execution_name,reasonable_count,part_reasonable_count,unreasonable_count,t_reason_name,contract_count,sum_count_purposes
0,11694128,0.0,7309900546,2019-09-23,120603.88,2019-09-23,2019-09-30,7327012462,7.0,"Электроэнергия, произведенная теплоэлектроцент...",NaN,0.0,0.0,0.0,0.0,9.0,0.0
1,10027697,0.0,7309900546,2019-01-11,219493.11,2019-01-01,2019-12-31,7327012462,364.0,"Электроэнергия, произведенная теплоэлектроцент...",NaN,0.0,0.0,0.0,0.0,9.0,0.0
2,10027697,0.0,7309900546,2019-01-11,219493.11,2019-01-01,2019-12-31,7327012462,364.0,"Электроэнергия, произведенная теплоэлектроцент...",NaN,0.0,0.0,0.0,0.0,9.0,0.0
3,14108652,0.0,7309900546,2019-12-18,17896.89,2019-10-01,2019-12-31,7315905278,91.0,Вода питьевая,NaN,0.0,0.0,0.0,0.0,9.0,0.0
4,14108652,0.0,7309900546,2019-12-18,17896.89,2019-10-01,2019-12-31,7315905278,91.0,Вода питьевая,NaN,0.0,0.0,0.0,0.0,9.0,0.0


In [19]:
okpd_group = contract_customer.groupby(by=['tru_okpd2_name', 'supplier_inn']).agg({'time_for_work': 'mean',
                                                                                  'reasonable_count': 'sum',
                                                                                   'part_reasonable_count': 'sum',
                                                                                   'unreasonable_count':'sum',
                                                                                   't_reason_name': 'sum',
                                                                                   'id_contract': 'count'})
okpd_group.head(40)

time_for_work  reasonable_count  \
tru_okpd2_name supplier_inn                                    
0              0                109.136364               0.0   
               1001000742       242.666667               0.0   
               100100219053     176.000000               0.0   
               100100798315     157.800000               2.0   
               1001012875       343.091503               0.0   
               1001013117       364.000000               0.0   
               100101430820     149.000000               8.0   
               100101439742      82.000000               0.0   
               1001014512       133.000000               0.0   
               100101539722     142.800000               0.0   
               1001020058       114.000000               0.0   
               1001023891       140.000000               0.0   
               100103148844      36.000000              56.0   
               1001041033       151.285714               4.0   
               1001041594       227.000000               0.0   
               100104331698     265.500000              32.0   
               1001047966        50.000000              56.0   
               100104842766     354.529412               9.0   
               1001055117       143.000000               0.0   
               1001058990       193.625000               9.0   
               1001062509       261.000000               2.0   
               100107566994      40.500000               0.0   
               1001079157       119.000000               6.0   
               1001085520        72.500000               0.0   
               100108606368      25.000000               0.0   
               1001088338        70.727273               0.0   
               1001091997       164.000000               4.0   
               1001094807       169.533333               4.0   
               1001096064        59.400000               0.0   
               1001117532       232.500000               0.0   
               1001134168       147.313725               4.0   
               1001137169       121.000000               0.0   
               1001138317       116.000000               0.0   
               1001141006       319.000000               0.0   
               1001144014         9.000000               0.0   
               1001145434       380.000000               0.0   
               1001146276       190.571429               0.0   
               100115465358     192.250000              24.0   
               1001156436       379.500000               0.0   
               1001157214       183.000000               0.0   

                             part_reasonable_count  unreasonable_count  \
tru_okpd2_name supplier_inn                                              
0              0                               0.0                 0.0   
               1001000742                      4.0                 8.0   
               100100219053                    0.0                 0.0   
               100100798315                    0.0                 0.0   
               1001012875                      0.0                 0.0   
               1001013117                      0.0                 0.0   
               100101430820                   18.0                 2.0   
               100101439742                    0.0                 0.0   
               1001014512                      0.0                 0.0   
               100101539722                    0.0                 0.0   
               1001020058                      0.0                 0.0   
               1001023891                      0.0                 0.0   
               100103148844                    0.0                48.0   
               1001041033                      0.0                 0.0   
               1001041594                      0.0                 0.0   
               100104331698                    0.0                16.0   
        

In [20]:
inn_list = pd.read_csv('hack_2023/inn_list.csv', sep=';')
participation_statistic = pd.read_csv('hack_2023/participation_statistic.csv', sep=';')
participation_statistic = participation_statistic[['participant_inn', 'procedure_qty', 'win_qty']]
participation_statistic = participation_statistic.rename(columns={'participant_inn':'inn'})

company_info = inn_list.merge(participation_statistic, on='inn', how='outer')
company_info['win_part, %'] = company_info['win_qty'] / company_info['procedure_qty'] * 100
company_info = company_info.drop('win_qty', axis=1)
company_info

,inn,procedure_qty,"win_part, %"
0,7719484339,1.0,0.000000
1,7719484339,10.0,20.000000
2,2703065260,45.0,8.888889
3,6664047737,NaN,NaN
4,7718605781,9.0,55.555556
...,...,...,...
274693,1660179980,3.0,33.333333
274694,1660179980,11.0,36.363636
274695,7606037185,10.0,50.000000
274696,7606037185,23.0,47.826087


In [32]:
okpd_group['win_part, %'] = [ 0 for i in range(len(okpd_group.index))]
company_info['win_part, %'] = company_info['win_part, %'].fillna(0).astype('int64')
company_info = company_info.loc[~ company_info['inn'].duplicated()]
small_okpd_group = okpd_group.loc[okpd_group.index[:50000]]
dict_inns = {}
inenes = list(company_info['inn']) 
values_percent = list(company_info['win_part, %'])
for i in range(len(name)):
    dict_inns[inenes[i]]=values_percent[i]
for okpds, inns in notebook.tqdm(okpd_group.index):
    if int(inns) in dict_inns.keys():
        okpd_group.loc[(okpds, inns), 'win_part, %'] = dict_inns[int(inns)]

  0%|          | 0/1106421 [00:00<?, ?it/s]

# Сделаем рейтинг для каждого ИНН

Параметры, которые непосредственно влияют на рейтинг:
1. Жалобы
2. Время исполения
3. Причины расторжения контракта
4. Выигранные тендеры/все тендеры

In [33]:
okpd_group['sum_count_purposes'] = (okpd_group['reasonable_count'] +
                  okpd_group['part_reasonable_count'] + 
                  okpd_group['unreasonable_count'] +
                  okpd_group['t_reason_name'])
okpd_group['good_reason_name, %'] = (1 - (okpd_group['part_reasonable_count'] + \
                  okpd_group['unreasonable_count'] + okpd_group['t_reason_name'])/okpd_group['sum_count_purposes'])*100

In [53]:
import random 
okpd_group['time_for_work, %'] =  (1 -okpd_group['time_for_work']/okpd_group['time_for_work'].max() )*100
okpd_group['count_contract, %'] = [random.randint(0, 100) for i in range(len(okpd_group))]

In [60]:
okpd_group['reit'] = 2.5*(okpd_group['good_reason_name, %'].fillna(0) +
                  okpd_group['time_for_work, %'].fillna(0)  + 
                  okpd_group['count_contract, %'].fillna(0) +
                  okpd_group['win_part, %'].fillna(0))/100

In [61]:
okpd_group['reit']

tru_okpd2_name                                                                                          supplier_inn
0                                                                                                       0               2.342898
                                                                                                        1001000742      2.783333
                                                                                                        100100219053    2.675000
                                                                                                        100100798315    6.488750
                                                                                                        1001012875      0.455678
                                                                                                                          ...   
Ящики металлические, специально предназначенные для хранения денег и документов, и аналогичные изделия  78115

In [62]:
result_okpd_group = okpd_group[['good_reason_name, %', 'time_for_work, %', 'count_contract, %', 'win_part, %', 'reit']]

In [72]:
result_okpd_group = result_okpd_group.fillna(0)
def reit_classifier(row):
    if row > 5:
        return 1
    else:
        return 0
result_okpd_group['reit_classif'] = result_okpd_group['reit'].apply(reit_classifier)

In [75]:
result_okpd_group = result_okpd_group[['reit', 'reit_classif']]
result_okpd_group

reit  \
tru_okpd2_name                                     supplier_inn             
0                                                  0             2.342898   
                                                   1001000742    2.783333   
                                                   100100219053  2.675000   
                                                   100100798315  6.488750   
                                                   1001012875    0.455678   
...                                                                   ...   
Ящики металлические, специально предназначенные... 7811506697    2.781250   
                                                   7816279649    1.262500   
                                                   782022535966  6.706250   
                                                   7839118567    6.296477   
                                                   784880081288  2.331250   

                                                                 reit_classif  
tru_okpd2_name                                     supplier_inn                
0                                                  0                        0  
                                                   1001000742               0  
                                                   100100219053             0  
                                                   100100798315             1  
                                                   1001012875               0  
...                                                                       ...  
Ящики металлические, специально предназначенные... 7811506697               0  
                                                   7816279649               0  
                                                   782022535966             1  
                                                   7839118567               1  
                                                   784880081288             0  

[1106421 rows x 2 columns]

In [74]:
import os  
os.makedirs('folder', exist_ok=True)  
result_okpd_group.to_csv('resresult_okpd_group.csv')

# Инфо о компаниях

In [ ]:
inn_list = pd.read_csv('hack_2023/inn_list.csv', sep=';')
msp_reestr = pd.read_csv('hack_2023/msp_reestr.csv', sep=';')
egrul_info = pd.read_csv('hack_2023/egrul_info.csv', sep=';')
egrul_licenses = pd.read_csv('hack_2023/egrul_licenses.csv', sep=';')
egrip_licenses = pd.read_csv('hack_2023/egrip_licenses.csv', sep=';')
avg_staff_qty = pd.read_csv('hack_2023/avg_staff_qty.csv', sep=';')

In [ ]:
participation_statistic = participation_statistic[['participant_inn', 'procedure_qty', 'win_qty']]
participation_statistic = participation_statistic.rename(columns={'participant_inn':'inn'})
egrul_info = egrul_info[['inn', 'registration_date']]
egrul_licenses = egrul_licenses[['inn', 'license_activity_type']]
egrip_licenses = egrip_licenses[['inn', 'license_activity_type']]
avg_staff_qty = avg_staff_qty[['inn', 'avg_staff_qty']]

In [ ]:
company_info = inn_list.merge(participation_statistic, on='inn', how='outer')
company_info = company_info.merge(egrul_info, on='inn', how='outer')
company_info = company_info.merge(egrul_licenses, on='inn', how='outer')
company_info = company_info.merge(egrip_licenses, on='inn', how='outer')
company_info = company_info.merge(avg_staff_qty, on='inn', how='outer')
company_info.head(20)

In [ ]:
import os  
os.makedirs('folder', exist_ok=True)  
company_info.to_csv('company_info.csv')